##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Neural machine translation with attention

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/text/nmt_with_attention">
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />
    View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/text/nmt_with_attention.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/nmt_with_attention.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/text/nmt_with_attention.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This notebook trains a sequence to sequence (seq2seq) model for Spanish to English translation. This is an advanced example that assumes some knowledge of sequence to sequence models.

After training the model in this notebook, you will be able to input a Spanish sentence, such as *"¿todavia estan en casa?"*, and return the English translation: *"are you still at home?"*

The translation quality is reasonable for a toy example, but the generated attention plot is perhaps more interesting. This shows which parts of the input sentence has the model's attention while translating:

<img src="https://tensorflow.org/images/spanish-english.png" alt="spanish-english attention plot">

Note: This example takes approximately 10 minutes to run on a single P100 GPU.

In [2]:
# Copyright 2021 Gerard Garcia

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

#     http://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
!pip install stanza

     |████████████████████████████████| 337kB 7.0MB/s 


In [ ]:
import tensorflow as tf

import matplotlib.pyplot as plt

import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
from google.colab import drive
import unicodedata
import re
import numpy as np
import os
import io
import time
import stanza
import pickle
from nltk.translate.bleu_score import sentence_bleu

stanza.download('es') # download Spanish model

2021-07-02 11:18:01 INFO: Downloading default packages for language: es (Spanish)...


## Download and prepare the dataset

We'll use a language dataset provided by http://www.manythings.org/anki/. This dataset contains language translation pairs in the format:

```
May I borrow this book?	¿Puedo tomar prestado este libro?
```

There are a variety of languages available, but we'll use the English-Spanish dataset. For convenience, we've hosted a copy of this dataset on Google Cloud, but you can also download your own copy. After downloading the dataset, here are the steps we'll take to prepare the data:

1. Add a *start* and *end* token to each sentence.
2. Clean the sentences by removing special characters.
3. Create a word index and reverse word index (dictionaries mapping from word → id and id → word).
4. Pad each sentence to a maximum length.

In [ ]:
# Converts the unicode file to ascii
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
                 if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
  w = unicode_to_ascii(w.lower().strip())

  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  # w = re.sub(r"([?.¡!,¿])", r" \1 ", w) #Interjection chars to \1

  # replacing everything with space except (a-z, A-Z, "-", "\", "{", "}")
  w = re.sub(r"[^a-zA-Z0-9\\{}<>-]+", " ", w)
  # Interjection chars to \1
  w = re.sub(r"([¡¿])", r" \1 ", w)
  # Set space between exclamation and words
  w = re.sub(r"([!])", r" !", w)
  # Set space between interrogation and words 
  w = re.sub(r"([?])", r" ? ", w)
  # Multiple spaces to one space
  w = re.sub(r'[" "]+', " ", w)

  w = w.strip()

  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  w = '<start> ' + w + ' <end>'
  return str(w)

# 1. Remove the accents
# 2. Clean the sentences
# 3. Return word pairs in the format: [SPANISH, LSE]

#Dataset format SENTENCE1\tSENTENCE2\n
def create_dataset(path, num_examples=None):
  lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

  word_pairs = [[preprocess_sentence(w) for w in line.split('\t')]
                for line in lines[:num_examples]]

  return zip(*word_pairs)

### Process stanza POSTagged Text

In [ ]:
def get_feats(feats_str, selected_feats=None):
  #Get the features processed from a string like (feat=value|...) to a list object
  feats_obj = {}
  if feats_str != None:
    feats = str(feats_str).split('|')
    for feat in feats:
      # print(feat)
      splitted = feat.split('=')
      key = splitted[0]
      value = splitted[1]
      if selected_feats == None or key in selected_feats:
        feats_obj[key] = value
  return feats_obj

def process_text(nlp_text):
  result = []
  for document in nlp_text:
    for sentence in document.sentences:
      sent = []
      for word in sentence.words:
        pos = word.upos
        if pos != "DET":
          if word.text == "<start>" or word.text == "<end>" or word.text == "<unk>":
            elem = {
                "text": word.text,
                "lemma": word.text
            }
          if pos == "PUNCT" or pos == "ADP":
            elem = {
                "text": word.text,
                "lemma": word.text,
                "upos": word.upos
            }
          elif pos == "NOUN":
            #Text and plural/singular
            feats = get_feats(word.feats, ["Number", "Gender"])
            elem = {
                "text": word.text,
                "lemma": word.lemma,
                "feats": feats,
                "upos": word.upos,
                "xpos": word.xpos
            }
          elif pos == "VERB" or pos == "AUX":
            #Text and person/tense/verbform
            feats = get_feats(word.feats, ["Person", "Tense", "VerbForm"])
            elem = {
                "text": word.text,
                "lemma": word.lemma,
                "feats": feats,
                "upos": word.upos,
                "xpos": word.xpos
            }
          elif pos == "ADV":
            elem = {
                "text": word.text,
                "lemma": word.lemma,
                "upos": word.upos
            }
          elif pos == "ADJ":
            feats = get_feats(word.feats, ["Gender", "Number"])
            elem = {
                "text": word.text,
                "lemma": word.lemma,
                "feats": feats,
                "upos": word.upos
            }
          else:
            feats = get_feats(word.feats)
            elem = {
                "text": word.text,
                "lemma": word.lemma,
                "feats": feats,
                "upos": word.upos
            }
          sent.append(elem)
      result.append(sent)
  return result

def get_lemma_sentences(sentences_postagged):
  #Return all sentences simplified with lemma words
  processed_inp_lang = []
  for sentence_postagged in sentences_postagged:
    sentence = ""
    for word in sentence_postagged:
      # print(word)
      if word['text'] == "<start>" or word['text'] == "<end>" or word['text'] == "<unk>":
        sentence += word["text"]+ " "
      else:
        sentence += word["lemma"]+ " "
    processed_inp_lang.append(sentence)
  return processed_inp_lang

In [ ]:
# en_sentence = u"May I borrow this book?"
# sp_sentence = u"¿Puedo tomar prestado este libro?"
# print(preprocess_sentence(en_sentence))
# print(preprocess_sentence(sp_sentence).encode('utf-8'))

In [ ]:
def tokenize(lang):
  #Assign a value for each different word in the dictionary
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

  return tensor, lang_tokenizer

def convert(lang, tensor):
  #Test function to check tokenization
  sentence = ""
  for t in tensor:
    if t != 0:
      # print(f'{t} ----> {lang.index_word[t]}')
      sentence = sentence + f"{lang.index_word[t]} "
  # print(sentence)
  return sentence


def load_dataset(path, lemmatize=True, num_examples=None):
  # creating cleaned input, output pairs
  inp_lang_pre, targ_lang  = create_dataset(path, num_examples)
  if lemmatize:
    in_docs = [stanza.Document([], text=d) for d in list(inp_lang_pre)]
    inp_postagged = nlp(in_docs)
    inp_postagged2 = process_text(inp_postagged)
    inp_lang = get_lemma_sentences(inp_postagged2)
  else:
    inp_lang = inp_lang_pre
  input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
  target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [ ]:
# def load_data_from_path(path):
#   inp_lang, targ_lang = pickle.load(open(path, 'rb'))
#   input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
#   target_tensor, targ_lang_tokenizer = tokenize(targ_lang)
#   return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [ ]:
# es, lse = create_dataset(os.path.dirname("/content/drive/MyDrive/TFG/")+"/lse_dictionary.txt")
# print(es[234],lse[234])

## Write the encoder and decoder model

Implement an encoder-decoder model with attention which you can read about in the TensorFlow [Neural Machine Translation (seq2seq) tutorial](https://github.com/tensorflow/nmt). This example uses a more recent set of APIs. This notebook implements the [attention equations](https://github.com/tensorflow/nmt#background-on-the-attention-mechanism) from the seq2seq tutorial. The following diagram shows that each input words is assigned a weight by the attention mechanism which is then used by the decoder to predict the next word in the sentence. The below picture and formulas are an example of attention mechanism from [Luong's paper](https://arxiv.org/abs/1508.04025v5). 

<img src="https://www.tensorflow.org/images/seq2seq/attention_mechanism.jpg" width="500" alt="attention mechanism">

The input is put through an encoder model which gives us the encoder output of shape *(batch_size, max_length, hidden_size)* and the encoder hidden state of shape *(batch_size, hidden_size)*.

Here are the equations that are implemented:

<img src="https://www.tensorflow.org/images/seq2seq/attention_equation_0.jpg" alt="attention equation 0" width="800">
<img src="https://www.tensorflow.org/images/seq2seq/attention_equation_1.jpg" alt="attention equation 1" width="800">

This tutorial uses [Bahdanau attention](https://arxiv.org/pdf/1409.0473.pdf) for the encoder. Let's decide on notation before writing the simplified form:

* FC = Fully connected (dense) layer
* EO = Encoder output
* H = hidden state
* X = input to the decoder

And the pseudo-code:

* `score = FC(tanh(FC(EO) + FC(H)))`
* `attention weights = softmax(score, axis = 1)`. Softmax by default is applied on the last axis but here we want to apply it on the *1st axis*, since the shape of score is *(batch_size, max_length, hidden_size)*. `Max_length` is the length of our input. Since we are trying to assign a weight to each input, softmax should be applied on that axis.
* `context vector = sum(attention weights * EO, axis = 1)`. Same reason as above for choosing axis as 1.
* `embedding output` = The input to the decoder X is passed through an embedding layer.
* `merged vector = concat(embedding output, context vector)`
* This merged vector is then given to the GRU

The shapes of all the vectors at each step have been specified in the comments in the code:

In [ ]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state=hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [ ]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # query hidden state shape == (batch_size, hidden size)
    # query_with_time_axis shape == (batch_size, 1, hidden size)
    # values shape == (batch_size, max_len, hidden size)
    # we are doing this to broadcast addition along the time axis to calculate the score
    query_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(query_with_time_axis) + self.W2(values)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [ ]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state, attention_weights

In [ ]:
def initModelsFromZero(vocab_inp_size, vocab_tar_size, embedding_dim, units, batch_size, example_input_batch):
  #ENCODER MODEL
  encoder = Encoder(vocab_inp_size, embedding_dim, units, batch_size)
  sample_hidden = encoder.initialize_hidden_state()
  sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
  print('Encoder output shape: (batch size, sequence length, units)', sample_output.shape)
  print('Encoder Hidden state shape: (batch size, units)', sample_hidden.shape)
  # ATTENTION LAYER
  attention_layer = BahdanauAttention(10)
  attention_result, attention_weights = attention_layer(sample_hidden, sample_output)
  print("Attention result shape: (batch size, units)", attention_result.shape)
  print("Attention weights shape: (batch_size, sequence_length, 1)", attention_weights.shape)
  #DECODER MODEL
  decoder = Decoder(vocab_tar_size, embedding_dim, units, batch_size)
  sample_decoder_output, _, _ = decoder(tf.random.uniform((batch_size, 1)),
                                        sample_hidden, sample_output)
  print('Decoder output shape: (batch_size, vocab size)', sample_decoder_output.shape)
  return encoder, decoder

In [ ]:
def loadModels(encoder_path, decoder_path):
  loaded_encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
  loaded_decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)
  attention_layer = BahdanauAttention(10)

  loaded_encoder.load_weights(encoder_path)
  loaded_decoder.load_weights(decoder_path)

  sample_hidden = loaded_encoder.initialize_hidden_state()
  sample_output, sample_hidden = loaded_encoder(example_input_batch, sample_hidden)
  sample_decoder_output, _, _ = loaded_decoder(tf.random.uniform((BATCH_SIZE, 1)), sample_hidden, sample_output)
  print('Encoder output shape: (batch size, sequence length, units)', sample_output.shape)
  print('Encoder Hidden state shape: (batch size, units)', sample_hidden.shape)
  print('Decoder output shape: (batch_size, vocab size)', sample_decoder_output.shape)

  # ATTENTION LAYER
  attention_result, attention_weights = attention_layer(sample_hidden, sample_output)
  print("Attention result shape: (batch size, units)", attention_result.shape)
  print("Attention weights shape: (batch_size, sequence_length, 1)", attention_weights.shape)
  
  return loaded_encoder, loaded_decoder

## Define the loss function

In [ ]:
def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

## Training

1. Pass the *input* through the *encoder* which return *encoder output* and the *encoder hidden state*.
2. The encoder output, encoder hidden state and the decoder input (which is the *start token*) is passed to the decoder.
3. The decoder returns the *predictions* and the *decoder hidden state*.
4. The decoder hidden state is then passed back into the model and the predictions are used to calculate the loss.
5. Use *teacher forcing* to decide the next input to the decoder.
6. *Teacher forcing* is the technique where the *target word* is passed as the *next input* to the decoder.
7. The final step is to calculate the gradients and apply it to the optimizer and backpropagate.

In [ ]:
@tf.function
def train_step(encoder, decoder, inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    # Pass input to encoder
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden
    # Set initial input to decoder
    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
      # Calculate loss value with input step and predictions
      loss += loss_function(targ[:, t], predictions)
      # using teacher forcing. Set new input to decoder with past predictions
      dec_input = tf.expand_dims(targ[:, t], 1)

    # Calculate average loss in step
    batch_loss = (loss / int(targ.shape[1]))
    
    # Optimize variables comparing them with loss value
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

### Load dataset

In [ ]:
path_to_model = "/content/drive/MyDrive/TFG/saved_models_optimal"
# save = False

if True:
  drive.mount('/content/drive')
  path_to_file = os.path.dirname("/content/drive/MyDrive/TFG/")+"/lse_train_data.txt"

  nlp = stanza.Pipeline('es') # initialize Spanish neural pipeline
  num_examples = None
  input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, True, num_examples)
  
  #Store dataset
  # if save:  
  #   pickle.dump([input_tensor, target_tensor], open(path_to_model + "/dataset_lse.pkl", "wb"))
  

  # Calculate max_length of the target tensors
  max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]
  # Creating training and validation
  input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.05, shuffle=True)


  # Show length
  print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))
  print(type(input_tensor), type(target_tensor), type(inp_lang), type(targ_lang))
  print("Input Language; index to word mapping")
  convert(inp_lang, input_tensor_train[0])
  print("Target Language; index to word mapping")
  convert(targ_lang, target_tensor_train[0])

### Hyperparameters declaration

In [ ]:
# Define the hidden params to the models
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 1    # 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE # Calculate how many steps will be
embedding_dim = 256
units = 512  # Number of weights
l_rate=0.0005
EPOCHS = 15
loss_stop_dif = 0.10
max_loss_val = 0.5
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

### Training process

In [ ]:
train = True
load = False # True makes error on training
pre_loss = 1
losses = []
if train:
  # Choose if load past model or start new model from zero
  if load:
    encoder, decoder = loadModels(path_to_model + '/encoder_trained_weights', path_to_model + '/decoder_trained_weights')
  else:
    encoder, decoder = initModelsFromZero(vocab_inp_size, vocab_tar_size, embedding_dim, units, BATCH_SIZE, example_input_batch)
  
  #Define optimizer
  optimizer = tf.keras.optimizers.Adam(learning_rate=l_rate)  # learning_rate=0.01
  loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
  
  # checkpoint_dir = './training_checkpoints_lse_newdata'
  # checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
  # checkpoint = tf.train.Checkpoint(optimizer=optimizer,
  #                                 encoder=encoder,
  #                                 decoder=decoder)


  for epoch in range(EPOCHS):
    start = time.time()
    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0
    # Iterate through the input data
    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
      # Pass the input and real output to trainer. Enc_hidden can be declarated in many ways
      batch_loss = train_step(encoder, decoder, inp, targ, enc_hidden)
      total_loss += batch_loss

      if batch % 100 == 0:
        print(f'Epoch {epoch+1} Batch {batch} Loss {batch_loss.numpy():.4f}')
    
    # saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 2 == 0:
      # checkpoint.save(file_prefix=checkpoint_prefix)
      encoder.save_weights(path_to_model + '/encoder_trained_weights', save_format='tf')
      decoder.save_weights(path_to_model + '/decoder_trained_weights', save_format='tf')
    
    average_loss=total_loss/steps_per_epoch
    losses.append(average_loss)
    # Average of loss in epoch
    print(f'Epoch {epoch+1} Loss {average_loss:.4f}')
    print(f'Time taken for 1 epoch {time.time()-start:.2f} sec')
    difference = pre_loss-average_loss
    print("Diferencia", difference.numpy(),"\n")
    if difference.numpy() < loss_stop_dif and average_loss < max_loss_val:
      break
    
    pre_loss = average_loss
  
  # Plotting the loss evolution
  plt.plot(losses)
  plt.ylabel('Loss Value')
  plt.show()
else:
  encoder, decoder = loadModels(path_to_model+'/encoder_trained_weights', path_to_model+'/decoder_trained_weights')

## Translate

* The evaluate function is similar to the training loop, except we don't use *teacher forcing* here. The input to the decoder at each time step is its previous predictions along with the hidden state and the encoder output.
* Stop predicting when the model predicts the *end token*.
* And store the *attention weights for every time step*.

Note: The encoder output is calculated only once for one input.

In [ ]:
def evaluate(sentence, encoder, decoder):
  attention_plot = np.zeros((max_length_targ, max_length_inp))
  # Prepare sentence for the model
  sentence = preprocess_sentence(sentence)
  # Get lemmatized words
  in_docs = [stanza.Document([], text=d) for d in list([sentence])]
  inp_postagged = nlp(in_docs)
  inp_postagged2 = process_text(inp_postagged)
  lemma_sentence = get_lemma_sentences(inp_postagged2)[0][:-1]
  print("lemma", lemma_sentence)
  # Tokenize words
  unknown_word_id = inp_lang.word_index.get("<unk>", -1)
  inputs = []
  for word in lemma_sentence.split(' '):
    inputs.append(inp_lang.word_index.get(word, unknown_word_id))
  
  
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''
  hidden = [tf.zeros((1, units))]
  # Pass inputs to encoder and zero hidden state
  enc_out, enc_hidden = encoder(inputs, hidden)
  # Encoder output hidden state is decoder input hidden state
  dec_hidden = enc_hidden
  # Init decoder input
  dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

  for t in range(max_length_targ):
    # Get predictions from decoder
    predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)

    # storing the attention weights to plot later on
    attention_weights = tf.reshape(attention_weights, (-1, ))
    attention_plot[t] = attention_weights.numpy()

    predicted_id = tf.argmax(predictions[0]).numpy()
    # Translate predicted tokens to words
    result += targ_lang.index_word[predicted_id] + ' '

    if targ_lang.index_word[predicted_id] == '<end>':
      return result, sentence, attention_plot

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence, attention_plot

In [ ]:
# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
  fig = plt.figure(figsize=(10, 10))
  ax = fig.add_subplot(1, 1, 1)
  ax.matshow(attention, cmap='viridis')

  fontdict = {'fontsize': 14}

  ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
  ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

  ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
  ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

  plt.show()

In [ ]:
def translate(sentences, encoder, decoder):
  for sent in sentences:
    result, sentence, attention_plot = evaluate(sent, encoder, decoder)
    print('Input:', sentence)
    print('Predicted translation:', result)
  return result
  # attention_plot = attention_plot[:len(result.split(' ')),
  #                                 :len(sentence.split(' '))]
  # plot_attention(attention_plot, sentence.split(' '), result.split(' '))

## Restore the latest checkpoint and test

In [ ]:
# restoring the latest checkpoint in checkpoint_dir
# checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
translate(['hace mucho frio.',
           'Cual es tu signo?',
           'Esta manzana es roja.',
           'esta es mi vida.',
           '¿De qué color es?'], encoder, decoder)

## Next steps

* [Download a different dataset](http://www.manythings.org/anki/) to experiment with translations, for example, English to German, or English to French.
* Experiment with training on a larger dataset, or using more epochs.


In [ ]:
total=0
for i in range(0,len(input_tensor_val)):
  reference_sentence = convert(targ_lang, target_tensor_val[i])
  reference_sentence = reference_sentence.replace("<start> ", "", 1)
  # reference_sentence = reference_sentence.replace("<end>", "", 2)
  reference = [
      str(reference_sentence).split()
    ]
  candidate_sentence = convert(inp_lang, input_tensor_val[i])
  candidate_sentence = candidate_sentence.replace("<start>", "", 2)
  candidate_sentence = candidate_sentence.replace("<end>", "", 2)
  candidate_sentence = str(translate([candidate_sentence], encoder, decoder)).split()
  # Remove ends
  # candidate_sentence = candidate_sentence[:-1]
  # reference_sentence = reference_sentence.replace(" <end>", "", 1)

  print("Check", reference_sentence, candidate_sentence)

  result = sentence_bleu(reference, candidate_sentence)
  total = result + total
  print('BLEU score -> {}\n'.format(result))

average = total/len(input_tensor_val)
print('Number of tests: ', len(input_tensor_val))
print('AVERAGE BLEU SCORE: ', average)